In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import sys
import cv2
import os
import csv
import numpy as np
from PIL import Image
from PyQt5 import QtCore, QtGui, QtWidgets,QtCore, QtGui
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLineEdit,QInputDialog, QApplication,QLabel, QHBoxLayout,QComboBox,QLineEdit
from PyQt5.QtCore import QDate, QTime, QDateTime, Qt

class DOWNWindow(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent, QtCore.Qt.Window) 
        self.build()
 
    def build(self):
        ico=QtGui.QIcon("ico.png")
        self.setWindowIcon(ico)
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        print(x,y)
        self.mainLayout = QtWidgets.QVBoxLayout()
        self.setGeometry(x, y, 300,300)
        self.setWindowTitle('РАСПОЗНАВАНИЕ')
        
        self.lbl0=QLabel(self)
        self.lbl0.setGeometry(20, 0, 200, 40)
        self.lbl0.setText("Выберете номер группы:") 
        self.lbl1=QLabel(self)
        self.lbl1.setGeometry(50,200,200,40)
        
        self.combo = QComboBox(self)
        self.combo.setGeometry(20, 50,200,40)
        self.combo.addItems(["№1", "№2","№3"]) 
        
        
        self.btn0 = QPushButton('загрузить видео', self)
        self.btn0.setGeometry(20,200,200,40)
        self.btn0.clicked.connect(self.download)        
        self.show() 
        
    def download(self):
        d, _ = QtWidgets.QFileDialog.getOpenFileName(None, 'Open File', options=QtWidgets.QFileDialog.DontUseNativeDialog)
        file = open(d, 'r')  
        file.close()
        if str(self.combo.currentText())=="№1":
            tr='trainer1.yml'
        if str(self.combo.currentText())=="№2":
            tr='trainer2.yml'
        if str(self.combo.currentText())=="№3":
            tr='trainer3.yml'

        path='C:/DB/Unknow'
        imagePaths = [os.path.join(path,f) for f in os.listdir(path)]   
        if imagePaths==[]:
            face_id=0
        else:
            for imagePath in imagePaths:
                face_id = int(os.path.split(imagePath)[-1].split(".")[1])+1
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.read(tr)
        cascadePath = "haarcascade_frontalface_default.xml"
        faceCascade = cv2.CascadeClassifier(cascadePath);

        font = cv2.FONT_HERSHEY_SIMPLEX
        id = 0
        if str(self.combo.currentText())=="№1":
            name="C:/DB/FIO/1.csv"
            name1="C:/DB/FIO/1.1.csv"
        if str(self.combo.currentText())=="№2":
            name="C:/DB/FIO/2.csv" 
            name1="C:/DB/FIO/1.2.csv"
        if str(self.combo.currentText())=="№3":
            name="C:/DB/FIO/3.csv"
            name1="C:/DB/FIO/1.3.csv"
        r=csv.reader(open(name,newline=''))
        lines=list(r)

            
        names = lines
        cam = cv2.VideoCapture(d)
        cam.set(3, 640) 
        cam.set(4, 480) 
        minW = 0.1*cam.get(3) 
        minH = 0.1*cam.get(4)
        c=0

        while True:
            ret, img =cam.read()
            img = cv2.flip(img, 1) 
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

            faces = faceCascade.detectMultiScale( 
                gray,
                scaleFactor = 1.2,
                minNeighbors = 5,
                minSize = (int(minW), int(minH)),
               )

            for(x,y,w,h) in faces:
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
                id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

                
                if (confidence < 10):
                    id = names[id]
                    confidence = "  {0}%".format(round(100 - confidence))
                else:
                    id = "unknown"
                    confidence = "  {0}%".format(round(100 - confidence))
                if id !="unknown":
                    with open(name1, 'a') as f:
                        writer = csv.writer(f)
                        writer.writerow(str(id))
                    cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
                    cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
                    
                else:
                    c+=1
                    cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
                    cv2.imwrite(os.path.join(path ,"lime."+str(c) + ".jpg"), gray[y:y+h,x:x+w])

            cv2.imshow('camera',img) 

            k = cv2.waitKey(10) & 0xff 
            if k == 27:
                break

        cam.release()
        cv2.destroyAllWindows()
        
    def close_window(self):
        self.close()
        self.parent().edit.setText(self.edit.text())
        
class BLOCWindow(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent, QtCore.Qt.Window) 
        self.build()
 
    def build(self):
        ico=QtGui.QIcon("ico.png")
        self.setWindowIcon(ico)
        appearance = self.palette()
        appearance.setColor(QtGui.QPalette.Normal, QtGui.QPalette.Window,QtGui.QColor("white"))
        self.setPalette(appearance)
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        print(x,y)
        self.mainLayout = QtWidgets.QVBoxLayout()
        self.setGeometry(x, y, 600,300)
        self.setWindowTitle('РАСПОЗНАВАНИЕ')
        
        self.lbl0=QLabel(self)
        self.lbl0.setGeometry(10, 30, 100, 25)
        self.lbl0.setText("группа:") 
        self.lbl0.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold))
        self.lbl0.setStyleSheet("color:#66B2FF;")
        
        self.line1=QLineEdit(self)
        self.line1.setGeometry(100, 30,100,25)
        
        '''
        self.combo = QComboBox(self)
        self.combo.setGeometry(100, 30,100,25)
        self.combo.addItems(["№1", "№2","№3"]) 
        self.combo.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold))
        pal = self.combo.palette()
        pal.setColor(QtGui.QPalette.Button, QtGui.QColor(255,255,255))
        self.combo.setPalette(pal)
        self.combo.setStyleSheet("color: #66B2FF")
        '''

        
        self.lbl1=QLabel(self)
        self.lbl1.setGeometry(50,120,200,100)
        self.lbl1.setFont(QtGui.QFont("Times", 10, QtGui.QFont.Bold))
        self.lbl1.setStyleSheet("color:#66B2FF;")
        
        
        self.btn0 = QPushButton('отметить', self)
        self.btn0.setGeometry(250,200,350,100)
        self.btn0.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #66B2FF;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #CCE5FF;} 
        ''')
        self.btn0.clicked.connect(self.raspoznat) 
        self.btn0.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold))
        self.show() 
        
    def raspoznat(self):       
        id1=[]
        id2=[]
        print(str(self.line1.text()))
        if str(self.line1.text())=='1':
            tr='trainer1.yml'
        if str(self.line1.text())=='2':
            tr='trainer2.yml'
        if str(self.line1.text())=='3':
            tr='trainer3.yml'
            
        path='C:/DB/Unknow'
        imagePaths = [os.path.join(path,f) for f in os.listdir(path)]   
        if imagePaths==[]:
            face_id=0
        else:
            for imagePath in imagePaths:
                face_id = int(os.path.split(imagePath)[-1].split(".")[1])+1
        recognizer = cv2.face.LBPHFaceRecognizer_create()
        recognizer.read(tr)
        cascadePath = "haarcascade_frontalface_default.xml"
        faceCascade = cv2.CascadeClassifier(cascadePath);

        font = cv2.FONT_HERSHEY_SIMPLEX
        id = 0
        if str(self.line1.text())==str('1'):
            name="C:/DB/FIO/1.csv"
            name1="C:/DB/FIO/1.1.csv"
        if str(self.line1.text())==str('2'):
            name="C:/DB/FIO/2.csv" 
            name1="C:/DB/FIO/1.2.csv"
        if str(self.line1.text())==str('3'):
            name="C:/DB/FIO/3.csv"
            name1="C:/DB/FIO/1.3.csv"
        r=csv.reader(open(name,newline=''))
        lines=list(r)
        flat_list = [item for sublist in lines for item in sublist]
        now = QDate.currentDate()    
        names = flat_list
        cam = cv2.VideoCapture(1)
        cam.set(3, 640) 
        cam.set(4, 480) 
        minW = 0.1*cam.get(3) 
        minH = 0.1*cam.get(4)

        while True:
            ret, img =cam.read()
            img = cv2.flip(img, 1) 
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

            faces = faceCascade.detectMultiScale( 
                gray,
                scaleFactor = 1.2,
                minNeighbors = 5,
                minSize = (int(minW), int(minH)),
               )
            c=0
            for(x,y,w,h) in faces:
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
                id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

                
                if (confidence < 9):
                    id = names[id]
                    confidence = "  {0}%".format(round(100 - confidence))
                else:
                    id = "unknown"
                    confidence = "  {0}%".format(round(100 - confidence))
                if id !="unknown":  
                    id2.append(id)
                    cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
                    cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
                    
                else:
                    c+=1
                    cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
                    cv2.imwrite(os.path.join(path ,str(now.toString(Qt.ISODate))+'.'+str(c)   + ".jpg"), gray[y:y+h,x:x+w])
                    
            cv2.imshow('camera',img) 

            k = cv2.waitKey(100) & 0xff 
            if k == 27:
                break       
        for i in id2:
            if i not in id1:
                id1.append(i)
        id1.insert(0,now.toString(Qt.ISODate))
        with open(name1, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(id1)

        q = '\n'.join(id1) 
        self.lbl1.setText(q)
            
        cam.release()
        cv2.destroyAllWindows()
        
    def close_window(self):
        self.close()
        self.parent().edit.setText(self.edit.text())
        
class SecondWindow(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent, QtCore.Qt.Window) 
        self.build()
 
    def build(self):
        ico=QtGui.QIcon("ico.png")
        self.setWindowIcon(ico)
        window=QtWidgets.QWidget()
        appearance = self.palette()
        appearance.setColor(QtGui.QPalette.Normal, QtGui.QPalette.Window,QtGui.QColor("white"))
        self.setPalette(appearance)
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2
        print(x,y)
        self.mainLayout = QtWidgets.QVBoxLayout()
        self.setGeometry(x, y, 600,300)
        self.setWindowTitle('НОВЫЙ ПОЛЬЗОВАТЕЛЬ')
        
        self.lbl0=QLabel(self)
        self.lbl0.setGeometry(10, 10, 100, 25)
        self.lbl0.setText("группа:") 
        self.lbl0.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold))
        self.lbl0.setStyleSheet("color:#66B2FF;")
        
        self.line1=QLineEdit(self)
        self.line1.setGeometry(100, 10,100,25)
        
        '''
        self.combo = QComboBox(self)
        self.combo.setGeometry(100, 10,100,25)
        self.combo.addItems(["№1", "№2","№3"]) 
        self.combo.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold))
        pal = self.combo.palette()
        pal.setColor(QtGui.QPalette.Button, QtGui.QColor(255,255,255))
        self.combo.setPalette(pal)
        self.combo.setStyleSheet("color: #66B2FF")
        
        '''

        
        self.lbl1=QLabel(self)
        self.lbl1.setGeometry(250, 10, 100, 25)
        self.lbl1.setText("имя:") 
        self.lbl1.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold))
        self.lbl1.setStyleSheet("color: #66B2FF;")
        
        self.line0=QLineEdit(self)
        self.line0.setGeometry(300,10,200,25)
        
        self.btn0 = QPushButton('добавить', self)
        self.btn0.setGeometry(0,50,300,270)
        self.btn0.setFont(QtGui.QFont("Times", 22, QtGui.QFont.Bold))
        self.btn0.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #66B2FF;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #CCE5FF;} 
        ''')
        self.btn0.clicked.connect(self.add_polz)
        
        self.btn1 = QPushButton('сохранить', self)
        self.btn1.setGeometry(300,50,300,270)
        self.btn1.setFont(QtGui.QFont("Times", 22, QtGui.QFont.Bold))
        self.btn1.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #66B2FF;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #CCE5FF;} 
        ''')
        self.btn1.clicked.connect(self.save_izm) 
        #app.setStyle("fusion")
        self.show()
        
    def add_polz(self):
        
        if str(self.line1.text())=='1':
            name=r'C:/DB/FIO/1.csv'
            path="C:/DB/Image/1"
        if str(self.line1.text())=='2':
            name=r'C:/DB/FIO/2.csv'
            path="C:/DB/Image/2"
        if str(self.line1.text())=='3':
            name=r'C:/DB/FIO/3.csv'
            path="C:/DB/Image/3"
            
        a=self.line0.text()
        a=a.split('/n')
        print(a)

        with open(name, 'a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(a)
            

        cam = cv2.VideoCapture(1)
        cam.set(3, 640) 
        cam.set(4, 480) 

        face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

        imagePaths = [os.path.join(path,f) for f in os.listdir(path)]   
        if imagePaths==[]:
            face_id=0
        else:
            for imagePath in imagePaths:
                face_id = int(os.path.split(imagePath)[-1].split(".")[1])+1
        count =0
        while(True):
            ret, img = cam.read()
            img = cv2.flip(img, 1) 
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_detector.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
                count += 1    
                cv2.imwrite(os.path.join(path ,"lime."+str(face_id) + '.' + str(count) + ".jpg"), gray[y:y+h,x:x+w])
                cv2.imshow('image', img)

            k = cv2.waitKey(100) & 0xff 
            if k == 27:
                break
            elif count >= 10: 
                 break
        cam.release()
        cv2.destroyAllWindows()  

    def save_izm(self):
        if str(self.line1.text())=="1":
            name='trainer1.yml'
            path="C:/DB/Image/1"
        if str(self.line1.text())=="2":
            name='trainer2.yml'
            path="C:/DB/Image/2"
        if str(self.line1.text())=="3":
            name='trainer3.yml'
            path="C:/DB/Image/3"
            

        recognizer = cv2.face.LBPHFaceRecognizer_create(1,8,4,4,123)
        detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

        def getImagesAndLabels(path):
            imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
            faceSamples=[]
            ids = []
            for imagePath in imagePaths:
                PIL_img = Image.open(imagePath).convert('L') 
                img_numpy = np.array(PIL_img,'uint8')
                id = int(os.path.split(imagePath)[-1].split(".")[1])
                faces = detector.detectMultiScale(img_numpy)
                for (x,y,w,h) in faces:
                    faceSamples.append(img_numpy[y:y+h,x:x+w])
                    ids.append(id)
            return faceSamples,ids

        faces,ids = getImagesAndLabels(path)
        recognizer.train(faces, np.array(ids))

        recognizer.write(name) 

        
    def close_window(self):
        self.close()
        self.parent().edit.setText(self.edit.text())
        
class Example(QWidget):
    def __init__(self):
        super().__init__()
        self.secondWin = None
        self.blocWin = None
        self.downWin = None
        self.initUI()

    def initUI(self):
        ico=QtGui.QIcon("ico.png")
        self.setWindowIcon(ico)
        
        window=QtWidgets.QWidget()
        desktop=QtWidgets.QApplication.desktop()
        x=(desktop.width()-window.width())//2
        y=(desktop.height()-window.height())//2

        self.mainLayout = QtWidgets.QVBoxLayout()
        self.setGeometry(x, y, 600,300)
        self.setWindowTitle('students')
        
        
        self.btn0 = QPushButton('add',self)
        self.btn0.setFont(QtGui.QFont("Times", 22, QtGui.QFont.Bold))
        self.btn0.setGeometry(0, 0,200,300)
        #self.btn0.setIcon(QIcon('d13.jpg'))
        #self.btn0.setIconSize(QSize(235, 900))
        self.btn0.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #66B2FF;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #CCE5FF;} 
        ''')
        self.btn0.clicked.connect(self.open_win1)
        
        
        self.btn1 = QPushButton('mark', self)
        self.btn1.setFont(QtGui.QFont("Times", 22, QtGui.QFont.Bold))
        self.btn1.setGeometry(200,0,200,300)
        self.btn1.clicked.connect(self.open_win2)
        self.btn1.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #66B2FF;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #CCE5FF;} 
        ''')
        
        self.btn2 = QPushButton('download', self)
        self.btn2.setFont(QtGui.QFont("Times", 22, QtGui.QFont.Bold))
        self.btn2.setGeometry(400,0,200,300)
        self.btn2.clicked.connect(self.open_win3)
        self.btn2.setStyleSheet('''
        QPushButton:hover {background-color: rgb(255, 255, 255); color: #66B2FF;} 
        QPushButton:!hover {background-color: rgb(255, 255, 255); color: #CCE5FF;} 
        ''')
        
        
        self.show()
    '''
        def showDialog(self):
        items = ("№1","№2","№3")
        item, okPressed = QInputDialog.getItem(self, "Выберете № группы","№:", items, 0, False)
        text1, ok = QInputDialog.getText(self, 'Input Dialog','Введите Ваше имя:')
       
                if ok:
            self.le1.setText(str(text1))
            self.le2.setText(str(text2))
      

    ''' 

        
    def open_win1(self):
        if not self.secondWin:
            self.secondWin = SecondWindow(self)
        self.secondWin.show()
        
    def open_win2(self):
        if not self.blocWin:
            self.blocWin = BLOCWindow(self)
        self.blocWin.show()
        
    def open_win3(self):
        if not self.downWin:
            self.downWin = DOWNWindow(self)
        self.downWin.show()
        
    def quitApp(self): 
        QtCore.QCoreApplication.instance().quit() 

        
if __name__ == '__main__':

    if QApplication.instance() != None: 
        app = QApplication.instance() 
    else: 
        app = QApplication(sys.argv) 
    ex = Example()
    app.exec_()

640 300
640 300
640 300
